In [1]:
import numpy as np
import pandas as pd
import json

In [2]:
df = pd.read_json('git_repos.json')
df

,total_count,incomplete_results,items
0,4588,False,"{'id': 28457823, 'node_id': 'MDEwOlJlcG9zaXRvc..."
1,4588,False,"{'id': 177736533, 'node_id': 'MDEwOlJlcG9zaXRv..."
2,4588,False,"{'id': 11730342, 'node_id': 'MDEwOlJlcG9zaXRvc..."
3,4588,False,"{'id': 13491895, 'node_id': 'MDEwOlJlcG9zaXRvc..."
4,4588,False,"{'id': 10270250, 'node_id': 'MDEwOlJlcG9zaXRvc..."
5,4588,False,"{'id': 45717250, 'node_id': 'MDEwOlJlcG9zaXRvc..."
6,4588,False,"{'id': 21737465, 'node_id': 'MDEwOlJlcG9zaXRvc..."
7,4588,False,"{'id': 2126244, 'node_id': 'MDEwOlJlcG9zaXRvcn..."
8,4588,False,"{'id': 60493101, 'node_id': 'MDEwOlJlcG9zaXRvc..."
9,4588,False,"{'id': 85077558, 'node_id': 'MDEwOlJlcG9zaXRvc..."


<bound method Series.items of total_count                                                        4588
incomplete_results                                                False
items                 {'id': 28457823, 'node_id': 'MDEwOlJlcG9zaXRvc...
Name: 0, dtype: object>